### **<p align="center">💥🎮Proyecto de recomendacion de videojuegos a los usuarios🎮💥</p>**

**<p align="center">Proceso de ETL (Extract, Transform and load) 🔍</p>**


Empezamos con una previa visualizacion de los datasets disponibilizados

importacion de librerias a utilizar:

In [4]:
import pandas as pd
import json
import ast

(Destacamos las anteriores como las principales librerias a utilizar pero no las unicas, a medida que avanzamos se pueden estar importando mas librerias)

- # Dataset N°1 steam games

Extraemos los datos del archivo json y visualizamos brevemente:

In [5]:
df_games = pd.read_json('output_steam_games.json', lines = True) 
df_games.head() 

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


conocemos el tamaño del archivo:

In [6]:
df_games.shape

(120445, 13)

Analizamos valores nulos: 

In [7]:
df_games.isnull().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

Eliminamos las filas con valores vacios:

In [8]:
df_games.dropna(how='all', inplace= True) #elimino filas donde todos los valores son nulos (NaN)
df_games.reset_index(drop=True, inplace= True) #restablezco los índices del DataFrame
print(df_games.shape)


(32135, 13)


visualizo nuevamente para ver como quedo

In [9]:

df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
4,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None


Verificamos la información general sobre los datos.

In [10]:
print(df_games.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.2+ MB
None


In [11]:
# verificamos los nulos nuevamente. 
df_games.isnull().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

En este punto no procedemos a eliminar los nulos para mas adelante explorar un analisis de estos ya que anteriormente eliminamos las filas que eran completamente nulas

Exportamos el data set en version csv:

In [12]:
df_games.to_csv('df_games.csv', index= True)

- # Dataset N°2 users items

Previamente visualizado el archivo users items, se observa que contiene errores de sintaxis y los datos se encuentran comprimidos. Explicaré paso a paso la metodologia para leer el archivo y trabajar con el a continuacion:  

- importamos bibliotecas para descomprimir el archivo:

In [13]:
import gzip

- creo e inicializo una lista vacia para almacenar los datos extraidos del archivo json y procedo con un bucle para descomprimir y procesar

In [14]:
data = []

for i in gzip.open('users_items.json.gz'):
    data.append(ast.literal_eval(i.decode('utf-8'))) #  hice uso de la libreria ast y la funcion literal_eval para convertir la cadena Json en un objeto Python

# a continuacion creo el DataFrame pandas
df_items = pd.DataFrame(data)

In [15]:
# realizo una previsualizacion:
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


- conozco el tamaño del DataFrame

In [16]:
df_items.shape

(88310, 5)

En este dataset se observa que la columna 'items' contiene datos anidados, los mismos son necesarios desanidarlos ya que contiene los Id y nombres de los videojuegos, los cuales son importantes para la recomendacion de estos a los usuarios 

- procedo a conocer exactamente el nombre de las columnas para asi, en el momento de desanidar la columna items, no tener problemas en mi codigo a la hora de llamarla.

In [17]:
df_items.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items'], dtype='object')

* Usamos la funcion explode ya que esta nos permite descomponer la columna y crear nuevas filas para cada elemento dentro de la columna items. Es decir, explotar las listas de la columna para tener filas independiente.
* Luego con la funcion json_normalize aplano los datos en un DataFrame

In [18]:
exp = df_items.explode('items')
df_desanidado = pd.json_normalize(exp['items'])

# elimino la columna anidad y reindexo la columna 'items' 
exp1 = exp.drop(columns='items').reset_index(drop=True)

# concateno al DataFrame desanidado
df_items = pd.concat([df_desanidado, exp1],axis=1)

print(df_items.shape)
df_items.head()

(5170015, 8)


,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


Analizo valores nulos :

In [19]:
df_items.isnull().sum()

item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
user_id                 0
items_count             0
steam_id                0
user_url                0
dtype: int64

Eliminamos valores nulos: 

In [20]:
df_desanidado.dropna(how='all',inplace=True)

In [21]:
df_items.isnull().sum()

item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
user_id                 0
items_count             0
steam_id                0
user_url                0
dtype: int64

elimino los nulos que se presentaban en la columna anidada

In [22]:
df_items.dropna(subset=['item_id','item_name','playtime_forever','playtime_2weeks'], inplace = True)

In [23]:
df_items.isnull().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
user_id             0
items_count         0
steam_id            0
user_url            0
dtype: int64

In [24]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 5170013
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   item_name         object 
 2   playtime_forever  float64
 3   playtime_2weeks   float64
 4   user_id           object 
 5   items_count       int64  
 6   steam_id          object 
 7   user_url          object 
dtypes: float64(2), int64(1), object(5)
memory usage: 353.8+ MB


- Exportamos a csv nombrandolo como df_items.csv

In [25]:
df_items.to_csv('df_items.csv', index= True)

- # Dataset N°3 user reviews

Este archivo, al igual que el anterior de items, trae errores de sintaxis, se encuentra comprimido y ademas trae datos anidados por lo que trabajare de manera similar al anterior explicando paso por paso:

* primero creamos una nueva lista que contenga los datos extraidos del json. Luego, creamos el DataFrame reviews

In [26]:
datareviews = []

for i in gzip.open('user_reviews.json.gz'):
    datareviews.append(ast.literal_eval(i.decode('utf-8'))) #uso la libreria ast y la funcion literal_eval para convertir la cadena Json en un objeto Python


df_reviews = pd.DataFrame(datareviews)

* visualizamos brevemente:

In [27]:
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


En este caso observamos que la columna reviews posee datos anidados por lo que sera necesario desanidarlos para mayor informacion y uso en el analisis del proyecto.

* conocemos el tamaño del dataset:

In [28]:
df_reviews.shape

(25799, 3)

In [29]:
# corroboramos el nombre de las columnas:
df_reviews.columns

Index(['user_id', 'user_url', 'reviews'], dtype='object')

* procedemos a desanidar la columna 'reviews' 

como primer paso hacemos uso de la funcion explode como en el DataFrame anterior para poder descomponer la columna y crear nuevas filas para cada elemento de los que se encuentran dentro de la columna reviews. Luego avanzamos con json_normalize 

In [30]:
exp = df_reviews.explode('reviews')
df_reviews_desanidado = pd.json_normalize(exp['reviews'])

#reindexo la columna 'user_id' de acuerdo a como descompuso la funcion explode
df1= exp ['user_id'].reset_index()

# y concateno la columna a df_reviews_desanidado:
df_reviews_desanidado['user_id'] = df1['user_id']

#por ultimo asigno el nombre correspondiente al df final como df_reviews
df_reviews = df_reviews_desanidado

#visualizamos
df_reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637


In [31]:
df_reviews.columns

Index(['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review', 'user_id'],
      dtype='object')

* Analizo la informacion del DF

In [32]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        59305 non-null  object
 1   posted       59305 non-null  object
 2   last_edited  59305 non-null  object
 3   item_id      59305 non-null  object
 4   helpful      59305 non-null  object
 5   recommend    59305 non-null  object
 6   review       59305 non-null  object
 7   user_id      59333 non-null  object
dtypes: object(8)
memory usage: 3.6+ MB


* Analizo valores nulos:

In [33]:
df_reviews.isnull().sum()

funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
user_id         0
dtype: int64

Analizamos los nulos a continuacion para eliminarlos de ser necesario:

In [34]:
filas_con_nulos = df_reviews[df_reviews['funny'].isnull()]
filas_con_nulos   

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gdxsd
177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198094224872
2559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198021575394
10080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cmuir37
13767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jaysteeny
15493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ML8989
19184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198079215291
20223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198079342142
25056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198061996985
26257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198108286351


Elimino estos valores nulos dado que frente al tamaño del DataFrame, 28 filas no afectan considerablemente el analisis del mismo ni el fin del proyecto

In [35]:
df_reviews.dropna(inplace = True)

In [36]:
df_reviews.isnull().sum()

funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
user_id        0
dtype: int64

* reviso si hay valores duplicados

In [37]:
print('total de valores duplicados: ', df_reviews.duplicated().sum())

total de valores duplicados:  874


* busco en que filas estan esos valores duplicados. 

In [38]:
duplicados = df_reviews[df_reviews.duplicated()]
print('filas duplicadas:')

duplicados

filas duplicadas:


,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
1114,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep,bokkkbokkk
2894,,"Posted January 10, 2014.",,218620,1 of 3 people (33%) found this review helpful,True,"Good graphics, fun heists! A bit laggy",ImSeriouss
2895,,"Posted January 10, 2014.",,105600,0 of 2 people (0%) found this review helpful,True,So fun! DEFINITELY NOT RIP OFF OF MINECRAFT! e...,ImSeriouss
2896,,"Posted December 17, 2014.",,570,No ratings yet,True,bobo pinoy,ImSeriouss
2897,,"Posted January 13, 2014.",,211820,No ratings yet,True,If you want to play this game.. expect glithes...,ImSeriouss
...,...,...,...,...,...,...,...,...
44456,,Posted July 3.,,422400,No ratings yet,True,Muy entretenido y una coleccion de armas prome...,76561198092022514
44457,,Posted June 1.,,218620,No ratings yet,True,"Tiene una jugabilidad y tematica muy buena :D,...",76561198092022514
44458,,"Posted August 17, 2014.",,261820,No ratings yet,True,"Buen juego, no importa el desarrrollo que tien...",76561198092022514
44459,,"Posted February 17, 2014.",,224260,No ratings yet,True,exelente aporte :D¡¡¡ es una buen mod basado e...,76561198092022514


Para eficiencia y reduccion del DataFrame eliminaremos las columnas consideradas como no necesarias: last_edited, helpful

In [39]:
df_reviews.drop(['last_edited', 'helpful'], axis=1, inplace=True)

In [40]:
df_reviews.head()

,funny,posted,item_id,recommend,review,user_id
0,,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637
4,,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637


* elimino las filas duplicadas:

In [41]:
df_reviews.drop_duplicates(inplace = True)

In [42]:
df_reviews.duplicated().sum()

0

Creare la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: 
* '0' si es malo 
* '1' si es neutral o si se encuentra ausente la reseña
* '2' si es positivo

In [43]:
# importacion de libreria necesaria.
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [44]:
#recurso necesario para nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [45]:
# Creo un objeto SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [46]:
# Funcion para asignar etiquetas segun la escala
def asignar_sentimiento(puntuacion):
    if puntuacion >= 0.05:
        return 2  # Positivo
    elif puntuacion <= -0.05:
        return 0  # Malo
    else:
        return 1  # Neutral o ausente

In [47]:
# Aplicacion de análisis de sentimiento y creacion de la nueva columna
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(lambda x: sia.polarity_scores(x)['compound'])
df_reviews['sentiment_analysis'] = df_reviews['sentiment_analysis'].apply(asignar_sentimiento)


In [48]:
df_reviews.head(10)

,funny,posted,item_id,recommend,review,user_id,sentiment_analysis
0,,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2
1,,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479,2
2,,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2
3,,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637,2
4,,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637,2
5,,"Posted November 29, 2013.",239030,True,Very fun little game to play when your bored o...,js41637,2
6,,Posted February 3.,248820,True,A suitably punishing roguelike platformer. Wi...,evcentric,2
7,,"Posted December 4, 2015.",370360,True,"""Run for fun? What the hell kind of fun is that?""",evcentric,2
8,,"Posted November 3, 2014.",237930,True,"Elegant integration of gameplay, story, world ...",evcentric,2
9,,"Posted October 15, 2014.",263360,True,"Random drops and random quests, with stat poin...",evcentric,1


In [49]:
# Elimino la columna original review
df_reviews.drop(['review'], axis=1, inplace=True)

* Estudio como se compone la columna posted para luego tratarla

In [50]:
df_reviews['posted'].describe()

count                     58431
unique                     1906
top       Posted June 21, 2014.
freq                        220
Name: posted, dtype: object

In [51]:
df_reviews['posted']

0         Posted November 5, 2011.
1            Posted July 15, 2011.
2           Posted April 21, 2011.
3            Posted June 24, 2014.
4        Posted September 8, 2013.
                   ...            
59328              Posted July 10.
59329               Posted July 8.
59330               Posted July 3.
59331              Posted July 20.
59332               Posted July 2.
Name: posted, Length: 58431, dtype: object

* creo una copia del DataFrame 'df_reviews' para hacer pruebas al momento de cambiar el formato de fecha sin afectar al original

In [52]:
df_copia = df_reviews.copy()

In [53]:
def convertir_fecha(fecha):
    try:
        # intenta extraer el ano y la fecha completa
        fecha_con_ano = pd.to_datetime(fecha, format='Posted %B %d, %Y.')
        return fecha_con_ano.strftime('%d/%m/%Y')
    except ValueError:
        try:
            # Si no tiene ano, intenta extraer solo la fecha
            fecha_sin_ano = pd.to_datetime(fecha, format='Posted %B %d.')
            return fecha_sin_ano.strftime('%d/%m')
        except ValueError:
            # Si no se puede extraer la fecha, devuelve la fecha original en formato dia/mes
            return fecha

# Aplica la función a la columna 'posted'
df_copia['new_date_format'] = df_copia['posted'].apply(convertir_fecha)


In [54]:
df_copia.head(10)

,funny,posted,item_id,recommend,user_id,sentiment_analysis,new_date_format
0,,"Posted November 5, 2011.",1250,True,76561197970982479,2,05/11/2011
1,,"Posted July 15, 2011.",22200,True,76561197970982479,2,15/07/2011
2,,"Posted April 21, 2011.",43110,True,76561197970982479,2,21/04/2011
3,,"Posted June 24, 2014.",251610,True,js41637,2,24/06/2014
4,,"Posted September 8, 2013.",227300,True,js41637,2,08/09/2013
5,,"Posted November 29, 2013.",239030,True,js41637,2,29/11/2013
6,,Posted February 3.,248820,True,evcentric,2,03/02
7,,"Posted December 4, 2015.",370360,True,evcentric,2,04/12/2015
8,,"Posted November 3, 2014.",237930,True,evcentric,2,03/11/2014
9,,"Posted October 15, 2014.",263360,True,evcentric,1,15/10/2014


In [55]:
df_copia.drop(['posted'], axis=1, inplace=True)

Dado que las transformaciones fueron realizadas con exito, se aplicaran los cambios en el DataFrame original:

In [56]:
df_reviews['posted'] = df_copia['new_date_format']

* A continuacion observamos que la columna 'posted' ahora contiene un formato dia/mes/año; a excepcion de aquellas fechas en las que el dato del año era incompleto. En estas el formato es dia/mes

In [57]:
df_reviews.head(10)

,funny,posted,item_id,recommend,user_id,sentiment_analysis
0,,05/11/2011,1250,True,76561197970982479,2
1,,15/07/2011,22200,True,76561197970982479,2
2,,21/04/2011,43110,True,76561197970982479,2
3,,24/06/2014,251610,True,js41637,2
4,,08/09/2013,227300,True,js41637,2
5,,29/11/2013,239030,True,js41637,2
6,,03/02,248820,True,evcentric,2
7,,04/12/2015,370360,True,evcentric,2
8,,03/11/2014,237930,True,evcentric,2
9,,15/10/2014,263360,True,evcentric,1


para almacenar lo trabajado, exporto el dataset en csv:

In [58]:
df_reviews.to_csv('df_reviews.csv', index= True)

convierto en parquet los archivos ya que son menos pesados y permiten trabajar mas facilmente.
(esto lo observe habiendo probado ya de ante mano trabajar con los archivos csv, por esta razon se converten ahora y no antes.)

In [131]:

df_games['price'] = pd.to_numeric(df_games['price'], errors='coerce')


In [133]:
ruta_games = 'proyecto-env\Data\df_games.csv'
ruta_games_parquet = 'games_parquet'

df_games.to_parquet(ruta_games_parquet, engine= 'pyarrow')

In [134]:
ruta_items = 'proyecto-env\Data\df_items.csv'
ruta_items_parquet = 'items_parquet'

df_items.to_parquet(ruta_items_parquet, engine= 'pyarrow')

In [135]:
ruta_reviews = 'proyecto-env\Data\df_reviews.csv'
ruta_reviews_parquet = 'reviews_parquet'

df_reviews.to_parquet(ruta_reviews_parquet, engine= 'pyarrow')

reviso los tamaños finales de cada DataFrame

In [66]:
print(df_games.shape)
print(df_games.columns)

(32135, 13)
Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')


In [65]:
print(df_items.shape)
print(df_items.columns)

(5153209, 8)
Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'user_id', 'items_count', 'steam_id', 'user_url'],
      dtype='object')


In [67]:
print(df_reviews.shape)
print(df_reviews.columns)

(58431, 6)
Index(['funny', 'posted', 'item_id', 'recommend', 'user_id',
       'sentiment_analysis'],
      dtype='object')
